In [ ]:
!pip install mysql-connector-python

In [ ]:
import pandas as pd
import mysql.connector

# Funzioni di servizio per la gestione della connessione

def connect(my_user, my_password):

  global mydb
  global mycursor

  try:
    mydb = mysql.connector.connect(host='90.147.166.237', user=my_user, password=my_password, database='db157')
  except mysql.connector.Error as err:
    print('Oops, qualcosa non ha funzionato')

  mycursor = mydb.cursor(buffered=True)

def close_connection():
  mycursor.close()
  mydb.close()

In [ ]:
# PROVA CONNESSIONE
connect("utente157", "d1r0mp3nt3")

In [ ]:
# OPERAZIONE 1 
def insert_pacchetto_viaggio(nome, proprietario_pacchetto, numerosita, fascia_prezzo, tipologia, data_inizio, data_fine, commento):
    query = "INSERT INTO Pacchetto_viaggio (nome, proprietario_pacchetto, numerosità, fascia_prezzo, tipologia, data_inizio, data_fine, commento) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    values = (nome, proprietario_pacchetto, numerosita, fascia_prezzo, tipologia, data_inizio, data_fine, commento)
    
    mycursor.execute(query, values)


In [ ]:
# PROVA OPERAZIONE 1
insert_pacchetto_viaggio('Dolomites', 'maria123', 4, 1000, 'Avventura', '2023-03-01', '2023-03-08', 'Exciting trip to the mountains')


In [ ]:
# OPERAZIONE 2
def cancella_utente(nickname):
    query = "DELETE FROM Utente WHERE nickname = %s"
    values = (nickname,)

    mycursor.execute(query, values)

In [ ]:
# PROVA OPERAZIONE 2 
cancella_utente("stefania_84")

In [ ]:
# OPERAZIONE 3
def recensioni_piu_alti():
    query = """
SELECT Recensione.proprietario_pacchetto AS "Proprietario Pacchetto", Recensione.pacchetto_recensito as "Pacchetto Viaggio", AVG(Recensione.punteggio) as Punteggio
FROM Recensione GROUP BY Recensione.pacchetto_recensito, Recensione.proprietario_pacchetto  ORDER BY Punteggio DESC
LIMIT 10
"""

    mycursor.execute(query)

    output = []

    for soggetto in mycursor:
      output.append(soggetto)

    df=pd.DataFrame(output, columns=['Proprietario Pacchetto', 'Pacchetto Viaggio', 'Punteggio'])
    return df

In [ ]:
# PROVA OPERAZIONE 3 
recensioni_piu_alti()

,Proprietario Pacchetto,Pacchetto Viaggio,Punteggio
0,giovanni99,Escursione alle isole greche,4.7
1,claudia90,Safari in Tanzania,4.5
2,Francesca6,Settimana al mare in Calabria,4.5
3,paolo_87,Tour della Toscana,4.5
4,massimo_83,Vacanze in montagna,4.2
5,federico333,Visita ai Musei di Parigi,4.0


In [ ]:
# OPERAZIONE 4 
def select_metodo_pagamento_luoghi_alloggio(nazione):
    query ="""
    SELECT 
    Metodo_pagamento.nome_utente AS Utente,
    Luoghi_alloggio.importo AS Importo,
    Luoghi_alloggio.nome AS "Luogo Alloggio"
FROM
    Metodo_pagamento
        JOIN
    Luoghi_alloggio ON Metodo_pagamento.numero_carta = Luoghi_alloggio.dati_pagamento
WHERE
    Luoghi_alloggio.nome IN (SELECT 
            Luoghi_alloggio.nome
        FROM
            Luoghi_alloggio
                JOIN
            Città ON Città.latitudine = Luoghi_alloggio.latitudine_city
                AND Città.longitudine = Luoghi_alloggio.longitudine_city
        WHERE
            Città.nazione = %s)
    """
    values = (nazione,)
    mycursor.execute(query, values)
    output = []

    for soggetto in mycursor:
      output.append(soggetto)

    df=pd.DataFrame(output, columns=['Utente', 'Importo', 'Luogo Alloggio'])
    return df


In [ ]:
# PROVA OPERAZIONE 4 
select_metodo_pagamento_luoghi_alloggio("Italy")

,Utente,Importo,Luogo Alloggio
0,gabriella444,150.25,Albergo del mare
1,gabriella444,250.00,Villa dei mari
2,gabriella444,250.50,Villa dei girasoli
3,claudia90,300.50,Villa delle colline
4,lucasito,200.25,Casa dei fiori


In [ ]:
# OPERAZIONE 5 
def trova_citta_evento(evento):
    query = """SELECT 
    Città.nome AS Città, Evento.nome_evento AS Evento, data_evento AS "Data Evento"
FROM
    Città
        JOIN
    Evento ON Evento.latitudine_city = Città.latitudine
        AND Evento.longitudine_city = Città.longitudine
WHERE
    Evento.tipologia LIKE CONCAT('%', %s, '%')"""

    values = (evento,)
    mycursor.execute(query, values)

    output = []

    for soggetto in mycursor:
      output.append(soggetto)

    df=pd.DataFrame(output, columns=['Citta', 'Nome Evento', 'Data Evento'])
    return df


In [ ]:
# PROVA OPERAZIONE 5
trova_citta_evento('Music')

,Citta,Nome Evento,Data Evento
0,Calabria,Calabria Music Festival,2023-07-15


In [ ]:
# OPERAZIONE 6 
def pacchetti_creati_e_acquisiti(nickname):
    query = """
SELECT DISTINCT
    Pacchetto_viaggio.nome AS "Pacchetto Viaggio",
    Acquisizione.proprietario_pacchetto AS "Proprietario Pacchetto"
FROM
    Pacchetto_viaggio
        LEFT JOIN
    Acquisizione ON Pacchetto_viaggio.nome = Acquisizione.nome_pacchetto
WHERE
    Pacchetto_viaggio.proprietario_pacchetto = %s
        OR Acquisizione.acquisitore = %s
    """
    values = (nickname,nickname, nickname)
    output = []

    mycursor.execute(query, values)

    for x in mycursor:
      output.append(x)

    df=pd.DataFrame(output, columns=['Pacchetti Creati e Acquisiti', 'Proprietario Pacchetto'])
    return df

In [ ]:
# PROVA OPERAZIONE 6
pacchetti_creati_e_acquisiti('claudia90')

,"Pacchetto Creato, Acquisito o Raccomandato all'Utente",Proprietario Pacchetto
0,Safari in Kenya,claudia90
1,Safari in Tanzania,claudia90
2,Tour della Toscana,paolo_87
3,Settimana al mare in Calabria,Francesca6


In [ ]:
# OPERAZIONE 7
def luoghi_piu_visitati(nazione):
    query = """
    SELECT Città.nome AS "Nome Città", Luoghi_visitati.nome AS "Luogo più visitato", COUNT(*) AS "Numero di visite"
    FROM Città
    JOIN (
      SELECT nome, latitudine_city, longitudine_city
      FROM Luoghi_visitati
    ) AS Luoghi_visitati
    ON Città.latitudine = Luoghi_visitati.latitudine_city AND Città.longitudine = Luoghi_visitati.longitudine_city
    WHERE Città.nazione = %s
    GROUP BY Città.nome, Luoghi_visitati.nome
    ORDER BY Città.nome, "Numero di visite" DESC;
    """
    
    values = (nazione,)
    output = []

    mycursor.execute(query, values)

    for x in mycursor:
      output.append(x)
    
    # Convert results to a pandas DataFrame and return it
    columns = [x[0] for x in mycursor.description]
    df = pd.DataFrame(output, columns=columns)
    return df

In [ ]:
# PROVA OPERAZIONE 7
luoghi_piu_visitati('Italy')

,Nome Città,Luogo più visitato,Numero di visite
0,Calabria,Reggia di Calabria,1
1,Sardegna,Costa Smeralda,1
2,Sardegna,La Maddalena Archipelago,1
3,Sardegna,Porto Cervo,1
4,Tuscany,Florence Cathedral,1
5,Tuscany,Torre del Mangia,1


In [ ]:
# OPERAZIONE 8 
def luoghi_visitati_nel_pacchetto(pacchetto):
    query = """
SELECT c.nome_pacchetto AS "Nome Pacchetto", lv.nome AS "Luogo Visitato", CONCAT(lv.orario_apertura," -- ",lv.orario_chiusura) AS "Orario Apertura - Chiusura"
FROM Luoghi_visitati lv
INNER JOIN Composizione c ON lv.civico = c.civico AND lv.cap = c.cap AND lv.via = c.via
WHERE c.nome_pacchetto = %s;
    """
    values = (pacchetto,)
    output = []

    mycursor.execute(query, values)

    for x in mycursor:
        output.append(x)

    df=pd.DataFrame(output, columns=['Nome Pacchetto', 'Luogo Visitato', 'Orario Apertura - Chiusura'])
    return df


In [ ]:
# PROVA OPERAZIONE 8
luoghi_visitati_nel_pacchetto('Viaggio a Rio de Janeiro')

,Nome Pacchetto,Luogo Visitato,Orario Apertura - Chiusura
0,Viaggio a Rio de Janeiro,Christ the Redeemer,09:00:00 -- 17:00:00
1,Viaggio a Rio de Janeiro,Copacabana Beach,10:00:00 -- 22:00:00


In [ ]:
# OPERAZIONE 9 
def trova_barriere_architettoniche_in_italia():
SELECT Luoghi_visitati.nome AS "Luogo Visitato", CONCAT (Luoghi_visitati.via,', ', Luoghi_visitati.civico,' , ' ,Luoghi_visitati.cap) AS Indirizzo 
FROM Luoghi_visitati JOIN Città ON Luoghi_visitati.longitudine_city = Città.longitudine AND Luoghi_visitati.latitudine_city = Città.latitudine
WHERE Città.nazione ='Italy' AND Luoghi_visitati.barriere_architettoniche = TRUE;
"""
    mycursor.execute(query)

    output=[]
    for x in mycursor:
      output.append(x)

    df=pd.DataFrame(output, columns=['Luogo Visitato', 'Indirizzo'])
    return df

In [ ]:
# PROVA OPERAZIONE 9
trova_barriere_architettoniche_in_italia()

,Luogo Visitato,Indirizzo
0,Torre del Mangia,"Piazza del Campo, 8 , 10002"


In [ ]:
# OPERAZIONE 10 
def pacchetti_acquisiti_e_gallerie(acquisitore):
    query = """
    SELECT Acquisizione.nome_pacchetto AS "Nome Pacchetto", Galleria.nome AS "Nome Galleria", Galleria.foto AS Foto, Galleria.commento AS Commento
FROM (
  SELECT nome_pacchetto, acquisitore
  FROM Acquisizione
  WHERE acquisitore = %s
) AS Acquisizione
INNER JOIN Galleria USING (nome_pacchetto);
    """
    values = (acquisitore,)
    output = []

    mycursor.execute(query, values)

    for soggetto in mycursor:
      output.append(soggetto)

    df=pd.DataFrame(output, columns=['Nome Pacchetto','Nome Galleria','Foto','Descrizione'])
    return df

In [ ]:
# PROVA OPERAZIONE 10
pacchetti_acquisiti_e_gallerie('Anna4')

,Nome Pacchetto,Nome Galleria,Foto,Descrizione
0,Visita ai Musei di Parigi,ciao12_musei_parigi_1,b'musei_parigi_foto1',La visita ai musei di Parigi è stata un esperi...
1,Visita ai Musei di Parigi,giovanni99_musei_parigi_2,b'musei_parigi_foto2',Ho scoperto molte opere d arte interessanti


In [ ]:
# OPERAZIONE 11 
def trova_specifica_localita(localita):
    query = """
SELECT Pacchetto_viaggio.nome AS "Nome Pacchetto", Pacchetto_viaggio.proprietario_pacchetto AS "Proprietario Pacchetto", COUNT(Acquisizione.acquisitore) AS "Numero Acquisizioni"
FROM Pacchetto_viaggio
JOIN Acquisizione ON Pacchetto_viaggio.nome = Acquisizione.nome_pacchetto AND Pacchetto_viaggio.proprietario_pacchetto = Acquisizione.proprietario_pacchetto
WHERE Pacchetto_viaggio.commento LIKE CONCAT('%', %s, '%')
GROUP BY Pacchetto_viaggio.nome, Pacchetto_viaggio.proprietario_pacchetto
ORDER BY COUNT(Acquisizione.acquisitore) DESC
    """
    values = (localita,)
    output = []

    mycursor.execute(query, values)

    for soggetto in mycursor:
      output.append(soggetto)

    df = pd.DataFrame(output, columns=['Nome Pacchetto', 'Proprietario pacchetto', 'Numero acquisizioni'])
    return df

In [ ]:
# PROVA OPERAZIONE 11
trova_specifica_localita("mare")

,Nome Pacchetto,Proprietario pacchetto,Numero acquisizioni
0,Settimana al mare in Calabria,Francesca6,2
1,Settimana al mare in Sardegna,Francesca6,1


In [ ]:
# OPERAZIONE 12 
def trova_pacchetti_algoritmo(ricevitore):
    query = """
SELECT p.nome AS "Pacchetto Viaggio Consigliato", p.proprietario_pacchetto AS "Proprietario Pacchetto", p.numerosità AS Numerosità, p.fascia_prezzo AS "Fascia Prezzo", p.tipologia AS Tipologia, CONCAT(p.data_inizio," -- ",p.data_fine) AS "Data Inizio - Fine"
FROM Pacchetto_viaggio p
INNER JOIN Algoritmo a ON p.nome = a.pacchetto_consigliato AND p.proprietario_pacchetto = a.proprietario_pacchetto
WHERE a.ricevitore = %s AND NOT EXISTS (
  SELECT * FROM Acquisizione acq WHERE acq.nome_pacchetto = p.nome AND acq.acquisitore = p.proprietario_pacchetto
);
    """
    values = (ricevitore,)
    output = []

    mycursor.execute(query, values)

    for soggetto in mycursor:
      output.append(soggetto)
   
    df = pd.DataFrame(output, columns=['Pacchetto Viaggio Consigliato', 'Proprietario Pacchetto', 'Numerosità', 'Fascia Prezzo', 'Tipologia', 'Data Inizio -- Fine'])
    return df

In [ ]:
# PROVA OPERAZIONE 12
trova_pacchetti_algoritmo("Giovanni5")

,Pacchetto Viaggio Consigliato,Proprietario Pacchetto,Numerosità,Fascia Prezzo,Tipologia,Data Inizio -- Fine
0,Tour della Toscana,paolo_87,3,600.0,culturale,2023-06-01 -- 2023-06-05
1,Vacanze in montagna,massimo_83,6,700.0,per famiglie,2023-12-20 -- 2023-12-27
2,Viaggio a New York,valentina_80,2,1000.0,avventura,2023-09-01 -- 2023-09-07


In [ ]:
# OPERAZIONE 13 
def utenti_piu_seguiti():
    query = """
SELECT utente, COUNT(Follow.utente_follower) AS "Numero Followers", AVG(TIMESTAMPDIFF(YEAR, Utente.data_nascita, NOW())) AS "Eta' media dei followers"
FROM Follow JOIN Utente ON Follow.utente_follower = Utente.nickname
GROUP BY utente
ORDER BY COUNT(Follow.utente_follower) DESC
LIMIT 10;
    """
    output = []

    mycursor.execute(query)

    for soggetto in mycursor:
      output.append(soggetto)


    df = pd.DataFrame(output, columns=['Utente', 'Numero di follower', "Eta' media dei followers"])
    return df


In [ ]:
# PROVA OPERAZIONE 13
utenti_piu_seguiti()

,Utente,Numero di follower,Eta' media dei followers
0,giovanni99,3,23.6667
1,davide78,2,31.5000
2,maria123,2,22.0000
3,federico333,2,34.0000
4,francesca456,2,22.5000
5,alessio111,2,26.5000
6,caterina222,2,29.0000
7,lucasito,2,21.0000
8,claudia90,2,28.5000
9,luciano_88,1,20.0000


In [ ]:
# OPERAZIONE 14
def pacchetti_con_min_n_acquisizioni(n):
    query = """
SELECT Pacchetto_viaggio.nome AS "Pacchetto Viaggio", Pacchetto_viaggio.proprietario_pacchetto AS "Proprietario Pacchetto", COUNT(Acquisizione.acquisitore) AS "Numero Acquisizioni"
    FROM Pacchetto_viaggio
    LEFT JOIN Acquisizione ON Pacchetto_viaggio.nome = Acquisizione.nome_pacchetto AND Pacchetto_viaggio.proprietario_pacchetto = Acquisizione.proprietario_pacchetto
    GROUP BY Pacchetto_viaggio.nome, Pacchetto_viaggio.proprietario_pacchetto
    HAVING COUNT(Acquisizione.acquisitore) < %s
    ORDER BY COUNT(Acquisizione.acquisitore) DESC
    """

    values = (n,)

    mycursor.execute(query, values)
    output = []

    for x in mycursor:
      output.append(x)

    df = pd.DataFrame(output, columns=["Nome Pacchetto", "Proprietario Pacchetto", "Numero di acquisizioni"])
    return df


In [ ]:
# PROVA OPERAZIONE 14
pacchetti_con_min_n_acquisizioni(2)

,Nome Pacchetto,Proprietario Pacchetto,Numero di acquisizioni
0,Viaggio a Berlino,paolo_87,1
1,Viaggio a Rio de Janeiro,massimo_83,1
2,Settimana al mare in Sardegna,Francesca6,1
3,Vacanze in Sardegna,claudia_90,0


In [ ]:
close_connection()